In [2]:
from unsupervised_topic_segmentation import core, eval
import pandas as pd
import numpy as np
import pickle

In [3]:
# read transcripts.pickle
with open('transcripts.pickle', 'rb') as handle:
    transcripts = pickle.load(handle)

In [ ]:
# options:
# core.TopicSegmentationDatasets.AMI, core.TopicSegmentationDatasets.ICSI
# core.TopicSegmentationAlgorithm.BERT, core.TopicSegmentationAlgorithm.RANDOM, core.TopicSegmentationAlgorithm.EVEN
# core.TopicSegmentationConfig [edit in unsupervised_topic_segmentation/types.py]